In [ ]:
from snowflake.snowpark import functions, Session, DataFrame
from snowflake.snowpark.types import StringType

In [ ]:
# Used by function create_session(...)
def create_key_pair_auth_objects(parconfigDir, parconfigFile) -> None:
    
    # Used when creating a session
    props = {}
    private_key_bytes = None
    
    # Path to the key/pair auth file
    configDir = parconfigDir #"/Users/pabloloza/Desktop/workshop/.ssh"
    configFile = parconfigFile #configDir + "/sf_config"
    
    # Load configuration file
    with open(configFile) as f:
        lines = f.readlines()
        
    # Convert configuration to a properties map
    for line in lines:
        (key, value) = line.split('=')
        props.update({key.lower() : value[0:-1]})
        
    # Convert the private key to a DER-encoded bytes object
    from cryptography.hazmat.primitives import serialization
    from cryptography.hazmat.backends import default_backend
    
    with open(props['private_key_file'], "rb") as key:
        private_key = serialization.load_pem_private_key(
            key.read(),
            password=None,
            backend=default_backend()
        )
    private_key_bytes = private_key.private_bytes(
        encoding=serialization.Encoding.DER,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    print(f"Read of key/pair authentication objects successful")
    return (props, private_key_bytes)
# end create_key_pair_auth_objects() 

<a id="cs"></a>

---

### Create A Session Object

The function `create_session()` will fail without the objects `props` and `private_key_bytes` that are created using function `create_key_pair_auth_objects()`. 

In [ ]:
from snowflake.snowpark import Session
def create_session(parconfigDir, parconfigFile) -> Session:
    try:
        (props,private_key_bytes) = create_key_pair_auth_objects(parconfigDir, parconfigFile)
    except Exception as ex:
        print(f"Could not create/read key/pair authentication objects: {str(ex)}")
        print(f"Is it possible that you have not already completed the key pair authentication exercise?")
        raise ex
    session = None
    try:
        session = (Session
                .builder      # props and private_key_bytes are created in function create_key_pair_auth_objects()
                .configs({**props, **{"private_key": private_key_bytes}})
                .create()
        )           
    except Exception as ex:
        print(f"Problem creating session: {str(ex)}")
        raise ex 
    print("User authenticated and session created")
    display_session_info(session)
    return session

---

<a id="util_functions"></a>

### Utility Functions

<a id="session_info"></a>

#### Session Information

In [ ]:

def get_account_from_session(sess:Session) -> str:
    return sess.get_current_account().strip('"')
#    return (sess.sql("SELECT CURRENT_ACCOUNT()")
#        .collect()[0][0]
#    )

# Session has no function current_user(). But the functions object does. 
def get_username_from_session(sess:Session) -> str:
    from snowflake.snowpark import functions
    return str(sess.create_dataframe([""]).select(functions.current_user()).collect()[0][0])

def get_user_from_session(sess:Session) -> str:
    return get_username_from_session(sess)

def get_user_name_from_session(sess:Session) -> str:
    return get_username_from_session(sess)

def username_from_session(sess:Session) -> str:
    return get_username_from_session(sess)

def get_snowflake_session_ID(sess:Session) -> str:
    from snowflake.snowpark import functions
    return (str(sess.create_dataframe([""])
        .select(
            functions.current_session()
         )
        .collect()[0][0]
      )
    )

def get_python_connector_session_ID(sess:Session) -> str:
    return get_session_info_item(sess, "python.connector.session.id")

def get_snowpark_version(sess:Session) -> str:
    return get_session_info_item(sess, "version")

def get_session_info_item(sess:Session, key:str) -> str:
    itemsArray = get_session_info_DF(sess).collect()
    for row in itemsArray:
        k = str(row[0])
        v = str(row[1])
        if(k == key):
            return v
    return None

def get_session_info_DF(sess:Session) -> DataFrame:
    # Retrieve the session info as text
    sess_info = str(sess._session_info)
    
    # Column objects used to parse the text
    new_line_col = functions.lit("\n")     # A Column holding a literal new line character
    value_col = functions.col("VALUE")
    colon_col = functions.lit(":")        # A Column holding a literal colon
    trim_chars_col = functions.lit(" \",") # A Column holding the characters to trim from keys and values
    
    # A Column with trimmed session information
    session_info_col = (functions          
            .trim(
                 functions.lit(sess_info)
                ,new_line_col
             )
    )
    
    # A table function object representing the Snowflake function SPLIT_TO_TABLE
    split_to_table_func = functions.table_function("split_to_table")
    
    # Create the session info DataFrame
    session_info_DF = (sess
     .table_function(
       split_to_table_func(
            session_info_col # Column to split
           ,new_line_col     # Split on a new line
        )              
     )
    .select(
        
        # First column selected
         functions.trim(
            functions.split(
                value_col          # Line to split
               ,colon_col          # Split on a colon (:)
             )[0]                 # First token from split
               .cast(StringType())# Cast to String
            ,trim_chars_col         # Trim spaces, commas, and quote marks
           )
         .alias("Session Property Name") # Rename column
        
        # Second column selected
        ,functions.trim(
            functions.split(
                value_col
               ,colon_col
             )[1]                    # Second token from split
               .cast(StringType())
            ,trim_chars_col
           )
         .alias("Session Property Value")
     )    
    )
    return session_info_DF
# end get_session_info_DF(Session)  

def display_session_info(sess:Session) -> None:
    get_session_info_DF(sess).show()
    
def print_session_info(sess:Session) -> None:
    print(str(sess._session_info))    

---

<a id="os_info"></a>

#### OS Information

In [ ]:
def get_os_env_variable(variable_name:str) -> str:
    import os
    return os.environ[variable_name]

def get_jupyter_hub_user_from_OS() -> str:
    return get_jupyter_hub_username_from_OS()

def get_jupyter_hub_user_name_from_OS() -> str:
    return get_jupyter_hub_username_from_OS()

def get_jupyter_hub_username_from_OS() -> str:
    return get_os_env_variable("JUPYTERHUB_USER")

def get_snowflake_account_from_OS() -> str:
    return get_jupyter_hub_user_from_OS().split("-")[0]

def get_snowflake_user_from_OS() -> str:
    return get_snowflake_username_from_OS()

def get_snowflake_user_name_from_OS() -> str:
    return get_snowflake_username_from_OS()

def get_snowflake_username_from_OS() -> str:
    return get_jupyter_hub_user_from_OS().split("-")[1]

def get_python_version() -> str:
    from platform import python_version
    return python_version()

def python_version() -> str:
    return get_python_version()

---

<a id="version_checking"></a>
#### Version Checking 

In [ ]:
def using_snowpark_version(sess:Session, version:str) -> bool:
    if(get_snowpark_version(sess) == version):
        return True
    return False

def using_python_version(version:str) -> bool:
    if(get_python_version() == version):
        return True
    return False

---

<a id="packages"></a>
#### Available Packages

Function `packages_dataframe` returns data from `INFORMATION_SCHEMA.PACKAGES`. By default, packages from all languages (Python, Scala, and Java) are included. Booleans can be passed to narrow the results. And a `count` can be customized as well. 

In [ ]:
def packages_dataframe(sess,python=True,java=True,scala=True,count=10) -> DataFrame:    
    sql_text = "SELECT * FROM INFORMATION_SCHEMA.PACKAGES WHERE "
    if(python):
        sql_text = sql_text + "LANGUAGE ILIKE 'python'"
        if(java):
            sql_text = sql_text + " OR "            
        
    if(java):
        sql_text = sql_text + "LANGUAGE ILIKE 'java'"
        if(scala):
            sql_text = sql_text + " OR "
        
    if(scala):
        sql_text = sql_text + "LANGUAGE ILIKE 'scala'"
        
    if((python==False) & (java==False) & (scala==False)):
        sql_text = sql_text + "FALSE"
    
    return sess.sql(sql_text)

def get_packages_dataframe(sess,python=True,java=True,scala=True,count=10) -> DataFrame:
    return packages_dataframe(sess,python=python,java=java,scala=scala,count=count)

def show_scala_packages(sess,count=10) -> None:
    packages_dataframe(sess,python=False,java=False,scala=True,count=count).show() 
    
def show_python_packages(sess,count=10) -> None:
    packages_dataframe(sess,python=True,java=False,scala=False,count=count).show()
    
def show_java_packages(sess,count=10) -> None:
    packages_dataframe(sess,python=False,java=True,scala=False,count=count).show()
    
def show_java_and_scala_packages(sess,count=10) -> None:
    packages_dataframe(sess,python=False,java=True,scala=True,count=count).show()       